In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from pymongo import MongoClient
import os
import re

# MongoDB 연결
MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGODB_ATLAS_CLUSTER_URI")
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)
DB_NAME = "test"
COLLECTION_NAME = "col"
collection = client[DB_NAME][COLLECTION_NAME]

def find_matching_words(text, language_from="KO", language_to="EN"):
    """
    텍스트에서 단어를 추출하고 MongoDB에서 매칭되는 단어들을 찾습니다.
    """
    # 텍스트에서 단어 추출
    words = re.findall(r'\w+', text)
    
    matched_translations = []
    for word in words:
        # MongoDB에서 단어 검색
        result = collection.find_one({f"metadata.{language_from}": word})
        if result:
            matched_translations.append({
                'original': word,
                'translation': result['metadata'][language_to]
            })
    
    return matched_translations

# 번역을 위한 PromptTemplate 설정
translation_prompt = PromptTemplate(
    input_variables=["dictionary_matches", "query"],
    template="""
다음은 번역에 참고할 수 있는 단어 사전입니다:
{dictionary_matches}

위 단어 사전을 참고하여, 아래 한국어 문장을 영어로 번역해주세요.
문장: {query}
번역:
"""
)

# LLM 체인 설정
trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
translation_chain = LLMChain(
    llm=trans_llm,
    prompt=translation_prompt
)

def translate_with_dictionary(query, from_lang="KO", to_lang="EN"):
    """
    MongoDB 사전을 활용하여 텍스트를 번역합니다.
    """
    # 매칭되는 단어들 찾기
    matches = find_matching_words(query, from_lang, to_lang)
    
    # 사전 매칭 결과를 문자열로 변환
    dictionary_matches = "\n".join([
        f"- {match['original']} → {match['translation']}"
        for match in matches
    ])
    
    if not dictionary_matches:
        dictionary_matches = "매칭되는 단어가 사전에 없습니다."
    
    # 번역 실행
    result = translation_chain.run({
        "dictionary_matches": dictionary_matches,
        "query": query
    })
    
    return {
        "translation": result,
        "matched_words": matches
    }
    

# 사용 예시
query = "산독끼는 바나나와 꼬라니를 좋아해."
result = translate_with_dictionary(query)

print("찾은 단어들:")
for match in result["matched_words"]:
    print(f"- {match['original']} → {match['translation']}")

print("\n최종 번역 결과:", result["translation"])

C:\Users\201\AppData\Local\Temp\ipykernel_8656\2544123652.py:50: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
C:\Users\201\AppData\Local\Temp\ipykernel_8656\2544123652.py:51: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  translation_chain = LLMChain(
C:\Users\201\AppData\Local\Temp\ipykernel_8656\2544123652.py:73: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = translation_chain.run({


찾은 단어들:

최종 번역 결과: The mountain rat likes bananas and corn.
